In [25]:
import os
import json
import shutil
import xml.etree.ElementTree as ET
import os

# Deal with the partnet mobility dataset
# Provide the path to the .json file and the folder you want to rename
partnet_mobility = 'assets/objects/partnet-mobility-v0/dataset'
target_folder_path = 'assets/objects'
if not os.path.exists(f"{target_folder_path}/obj_categories.json"):
    obj_categories = {}
else:
    with open(f'{target_folder_path}/obj_categories.json', 'r') as json_file:
        obj_categories = json.load(json_file)

for name in os.listdir(partnet_mobility):
    origin_folder_path = os.path.join(partnet_mobility, name)
    if os.path.isdir(origin_folder_path):
        json_file_path = os.path.join(origin_folder_path, 'result.json')
        with open(json_file_path, 'r') as json_file:
            data = json.load(json_file)
        if isinstance(data, list):
            data = data[0]
        if 'name' in data:
            obj_name = data['name']
            if obj_name in obj_categories:
                obj_categories[obj_name] += 1
            else:
                obj_categories[obj_name] = 0

            new_folder_path = os.path.join(target_folder_path, obj_name, str(obj_categories[obj_name]))
            os.makedirs(new_folder_path, exist_ok=True)
            os.rename(origin_folder_path, new_folder_path)
            print(f"Folder renamed to: {new_folder_path}")
        else:
            raise Exception(f"{json_file_path} has no name attributes.")
    else:
        raise Exception(f"{origin_folder_path} Folder not found.")
    
if not os.path.exists(f"{target_folder_path}/obj_categories.json"):
    with open(f'{target_folder_path}/obj_categories.json', 'w') as json_file:
        json.dump(obj_categories, json_file, indent=4)
else:
    print(f"File {target_folder_path}/obj_categories.json already exists.")

if os.path.exists(partnet_mobility):
    shutil.rmtree(os.path.dirname(partnet_mobility))

FileNotFoundError: [Errno 2] No such file or directory: 'assets/objects/partnet-mobility-v0/dataset'

In [24]:
# Deal with the YCB dataset
ycb_dataset_folder = 'assets/objects/partnet_mobility_folder'
target_folder_path = 'assets/objects'
obj_categories = {}


In [34]:
# Zip all objects folder and upload or download

In [ ]:
# Need to walk though all the folders to update obj_categories.json

 ```Given obj_name, urdf, mesh folder, it should create an object folder into the object dataset and update the .json file```

In [35]:
def update_urdf_file_mesh_path(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Define the new file paths
    new_collision_mesh_path = "textured_objs/collision.obj"
    new_textured_mesh_path = "textured_objs/textured.obj"

    # Update collision mesh file paths
    for collision in root.findall(".//collision"):
        for geometry in collision.findall(".//geometry"):
            for mesh in geometry.findall(".//mesh"):
                original_file_path = mesh.attrib["filename"]
                mesh.attrib["filename"] = new_collision_mesh_path

    # Update textured mesh file paths
    for visual in root.findall(".//visual"):
        for geometry in visual.findall(".//geometry"):
            for mesh in geometry.findall(".//mesh"):
                original_file_path = mesh.attrib["filename"]
                mesh.attrib["filename"] = new_textured_mesh_path

    # Save the modified URDF file
    tree.write(file_path, encoding='utf-8', xml_declaration=True)

def create_target_folder(obj_name, urdf_file_path, mesh_folder_path, pool_folder_path):
    target_folder_path = os.path.join(pool_folder_path, obj_name)
    if os.path.exists(target_folder_path):
        key = input(f"Folder {target_folder_path} already exists. \
                    Press d to delete whole folder, Press c to continue adding files, Press q to skip")
        if key == 'd':
            shutil.rmtree(target_folder_path)
        elif key == 'c':
            pass
        elif key == 'q':
            return
        else:
            raise Exception("Please rename the folder and try again.")

    os.makedirs(target_folder_path, exist_ok=True)
    obj_index = len(os.listdir(target_folder_path)) # Start from 0

    os.makedirs(os.path.join(target_folder_path, str(obj_index)), exist_ok=True)
    shutil.copyfile(urdf_file_path, os.path.join(target_folder_path, str(obj_index), 'mobility.urdf'))
    update_urdf_file_mesh_path(os.path.join(target_folder_path, str(obj_index), 'mobility.urdf'))
    if os.path.exists(mesh_folder_path):
        shutil.copytree(mesh_folder_path, os.path.join(target_folder_path, str(obj_index), 'textured_objs'))
    else:
        print(f"Mesh folder {mesh_folder_path} not found.")


In [36]:
import glob

obj_name = 'test'
urdf_folder_path = ""
mesh_folder_path = ""
pool_folder_path = 'assets/objects/ycb_objects_origin_at_center_vhacd'
urdf_files = glob.glob(os.path.join(pool_folder_path, "urdfs",  "*.urdf"))
obj_names = [os.path.basename(urdf_file).split(sep=".")[0] for urdf_file in urdf_files]

for obj_name in obj_names:
    urdf_file_path = os.path.join(pool_folder_path, "urdfs", f"{obj_name}.urdf")
    mesh_folder_path = os.path.join(pool_folder_path, "meshes", obj_name)
    create_target_folder(obj_name, urdf_file_path, mesh_folder_path, pool_folder_path)


Mesh folder assets/objects/ycb_objects_origin_at_center_vhacd/meshes/bin not found.
